In [1]:
from gensim.models import AuthorTopicModel

model = AuthorTopicModel.load('model.atmodel')

In [2]:
for topic in model.show_topics(num_topics=100):
    words = ''
    for word, prob in model.show_topic(topic[0]):
        words += word + ' '
    print('Words: ' + words)
    print()

Words: islámico irak comienza galería Estado Islámico gómez terrorismo tomó felipe monterrey 

Words: islámico irak comienza galería Estado Islámico gómez terrorismo tomó felipe monterrey 

Words: islámico irak comienza galería Estado Islámico gómez terrorismo tomó felipe monterrey 

Words: islámico irak comienza galería Estado Islámico gómez terrorismo tomó felipe monterrey 

Words: y a el o vida mundo escribe semana historia julio 

Words: islámico irak comienza galería Estado Islámico gómez terrorismo tomó felipe monterrey 

Words: islámico irak comienza galería Estado Islámico gómez terrorismo tomó felipe monterrey 

Words: islámico irak comienza galería Estado Islámico gómez terrorismo tomó felipe monterrey 

Words: islámico irak comienza galería Estado Islámico gómez terrorismo tomó felipe monterrey 

Words: islámico irak comienza galería Estado Islámico gómez terrorismo tomó felipe monterrey 

Words: islámico irak comienza galería Estado Islámico gómez terrorismo tomó felipe mon

In [3]:
model.show_topics(num_topics=100)

[(0,
  '0.000*"islámico" + 0.000*"irak" + 0.000*"comienza" + 0.000*"galería" + 0.000*"Estado Islámico" + 0.000*"gómez" + 0.000*"terrorismo" + 0.000*"tomó" + 0.000*"felipe" + 0.000*"monterrey"'),
 (1,
  '0.000*"islámico" + 0.000*"irak" + 0.000*"comienza" + 0.000*"galería" + 0.000*"Estado Islámico" + 0.000*"gómez" + 0.000*"terrorismo" + 0.000*"tomó" + 0.000*"felipe" + 0.000*"monterrey"'),
 (2,
  '0.000*"islámico" + 0.000*"irak" + 0.000*"comienza" + 0.000*"galería" + 0.000*"Estado Islámico" + 0.000*"gómez" + 0.000*"terrorismo" + 0.000*"tomó" + 0.000*"felipe" + 0.000*"monterrey"'),
 (3,
  '0.000*"islámico" + 0.000*"irak" + 0.000*"comienza" + 0.000*"galería" + 0.000*"Estado Islámico" + 0.000*"gómez" + 0.000*"terrorismo" + 0.000*"tomó" + 0.000*"felipe" + 0.000*"monterrey"'),
 (4,
  '0.179*"y" + 0.103*"a" + 0.050*"el" + 0.017*"o" + 0.011*"vida" + 0.009*"mundo" + 0.009*"escribe" + 0.009*"semana" + 0.008*"historia" + 0.008*"julio"'),
 (5,
  '0.000*"islámico" + 0.000*"irak" + 0.000*"comienza" + 

In [4]:
%%time
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=0)
smallest_author = 0  # Ignore authors with documents less than this.
authors = [model.author2id[a] for a in model.author2id.keys() if len(model.author2doc[a]) >= smallest_author]
_ = tsne.fit_transform(model.state.gamma[authors, :])  # Result stored in tsne.embedding_

CPU times: user 796 ms, sys: 20 ms, total: 816 ms
Wall time: 616 ms


In [5]:

from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [6]:
from bokeh.models import HoverTool
from bokeh.plotting import figure, show, ColumnDataSource

x = tsne.embedding_[:, 0]
y = tsne.embedding_[:, 1]
author_names = [model.id2author[a] for a in authors]

scale = 0.01
author_sizes = [len(model.author2doc[a]) for a in author_names]
radii = [size * scale for size in author_sizes]

source = ColumnDataSource(
        data=dict(
            x=x,
            y=y,
            author_names=author_names,
            author_sizes=author_sizes,
            radii=radii,
        )
    )

hover = HoverTool(
        tooltips=[
        ("author", "@author_names"),
        ("size", "@author_sizes"),
        ]
    )

p = figure(tools=[hover, 'crosshair,pan,wheel_zoom,box_zoom,reset,save,lasso_select'])
p.scatter('x', 'y', radius='radii', source=source, fill_alpha=0.6, line_color=None)
show(p)

In [7]:
from gensim.similarities import MatrixSimilarity


index = MatrixSimilarity(model[list(model.id2author.values())])



In [8]:

from gensim import matutils
import pandas as pd


author_vecs = [model.get_author_topics(author) for author in model.id2author.values()]

def similarity(vec1, vec2):
    dist = matutils.hellinger(matutils.sparse2full(vec1, model.num_topics), \
                              matutils.sparse2full(vec2, model.num_topics))
    sim = 1.0 / (1.0 + dist)
    return sim

def get_sims(vec):
    sims = [similarity(vec, vec2) for vec2 in author_vecs]
    return sims

def get_table(name, top_n=10, smallest_author=1):
    sims = get_sims(model.get_author_topics(name))


    table = []
    for elem in enumerate(sims):
        author_name = model.id2author[elem[0]]
        sim = elem[1]
        author_size = len(model.author2doc[author_name])
        if author_size >= smallest_author:
            table.append((author_name, sim, author_size))
            

    df = pd.DataFrame(table, columns=['Author', 'Score', 'Size'])
    df = df.sort_values('Score', ascending=False)[:top_n]
    
    return df

In [13]:
get_table('el_pais',top_n=136)

,Author,Score,Size
87,el_pais,1.000000,835
37,Letras_Libres,0.955727,171
48,PabloHiriart,0.950397,22
22,Esmas_com,0.945640,97
65,TapiaFernanda,0.925793,145
19,ElHuffPost,0.913811,947
6,AztecaNoticias,0.905944,254
117,mileniotv,0.905913,187
20,El_Universal_Mx,0.904904,1130
67,UnoNoticias,0.903284,516
